In [1]:
%autosave 3600

Autosaving every 3600 seconds


In [2]:
import bel_konzept as bk
import pandas as pd
import numpy as np

In [3]:
df_whg = pd.DataFrame(
    [('Schlafen',13.47,'zu'),
     ('Bad',9.06,'ab',45),
     ('Abstellraum',2.57,'ab',25),
     ('Kochen',7.16,'ab',45),
     ('Wohnen',26.98,'zu'),
     ('Zimmer_2',13.96,'zu'),
     ('Zimmer_1',11.89,'zu'),
     ('WC',1.45,'ab',25),
     ('Flur',9.07,'--')
    ],
    columns=['Raum','Flaeche','Typ','dV_min']
)
df_whg

,Raum,Flaeche,Typ,dV_min
0,Schlafen,13.47,zu,NaN
1,Bad,9.06,ab,45.0
2,Abstellraum,2.57,ab,25.0
3,Kochen,7.16,ab,45.0
4,Wohnen,26.98,zu,NaN
5,Zimmer_2,13.96,zu,NaN
6,Zimmer_1,11.89,zu,NaN
7,WC,1.45,ab,25.0
8,Flur,9.07,--,NaN


Nutzfläche in $m^2$

In [4]:
A_N = df_whg.Flaeche.sum()
A_N

95.61000000000001

Gesamter Zuluftvolumenstrom nach DIN 1946-6

In [5]:
dV_ges_zu = bk.dV_ges_zu(A_N)
dV_ges_zu # Gesamter Zuluftvolumenstrom in m**3/h

120.8102279

Gesamter Abluftvolumenstrom nach DIN 1946-6

In [6]:
dV_ges_ab = df_whg[df_whg.Typ == 'ab'].dV_min.sum()
dV_ges_ab # Gesamter Abluftvolumenstrom in m**3/h

140.0

Gesamter Volumenstrom

In [7]:
dV_ges=max(dV_ges_zu,dV_ges_ab)
dV_ges

140.0

Infiltrationsvolumenstrom nach DIN 1946-6

In [8]:
f_inf = 0.4 # Mehrgeschossig, Installationsschacht
n_50 = 1 # h**(-1)
h = 2.50 # m
delta_p = 5 # Mehrgeschossig, Windschwach

dV_inf = bk.dV_inf(f_inf,A_N,h,n_50,delta_p)
dV_inf # Infiltrationsvolumenstrom in m**3/h

20.58274616569973

Lüftungsvolumenstrom nach DIN 1946-6

In [9]:
dV_L = dV_ges - dV_inf
dV_L

119.41725383430027

In [10]:
df_whg['f'] = pd.DataFrame([2,3,2,2],index=df_whg[df_whg.Typ=='zu'].index)
df_whg

,Raum,Flaeche,Typ,dV_min,f
0,Schlafen,13.47,zu,NaN,2.0
1,Bad,9.06,ab,45.0,NaN
2,Abstellraum,2.57,ab,25.0,NaN
3,Kochen,7.16,ab,45.0,NaN
4,Wohnen,26.98,zu,NaN,3.0
5,Zimmer_2,13.96,zu,NaN,2.0
6,Zimmer_1,11.89,zu,NaN,2.0
7,WC,1.45,ab,25.0,NaN
8,Flur,9.07,--,NaN,NaN


In [11]:
i_zu = df_whg[df_whg.Typ=='zu'].index
i_ab = df_whg[df_whg.Typ=='ab'].index
f_ges = df_whg.f.sum()
dV_ges_ab
(i_zu,i_ab,f_ges,dV_ges_ab)

(Int64Index([0, 4, 5, 6], dtype='int64'),
 Int64Index([1, 2, 3, 7], dtype='int64'),
 9.0,
 140.0)

In [12]:
f_ges = df_whg.f.sum()
df_whg['dV_zu'] = pd.DataFrame(df_whg.f/f_ges*dV_L,index=i_zu)
df_whg['dV_ab'] = pd.DataFrame(df_whg.dV_min/dV_ges_ab*dV_L,index=i_ab)
df_whg

,Raum,Flaeche,Typ,dV_min,f,dV_zu,dV_ab
0,Schlafen,13.47,zu,NaN,2.0,26.537168,NaN
1,Bad,9.06,ab,45.0,NaN,NaN,38.384117
2,Abstellraum,2.57,ab,25.0,NaN,NaN,21.324510
3,Kochen,7.16,ab,45.0,NaN,NaN,38.384117
4,Wohnen,26.98,zu,NaN,3.0,39.805751,NaN
5,Zimmer_2,13.96,zu,NaN,2.0,26.537168,NaN
6,Zimmer_1,11.89,zu,NaN,2.0,26.537168,NaN
7,WC,1.45,ab,25.0,NaN,NaN,21.324510
8,Flur,9.07,--,NaN,NaN,NaN,NaN


In [13]:
df_whg['dV'] = df_whg[['dV_zu','dV_ab']].fillna(0).max(axis=1)
df_whg.drop(columns=['dV_zu','dV_ab'],inplace=True)
df_whg

,Raum,Flaeche,Typ,dV_min,f,dV
0,Schlafen,13.47,zu,NaN,2.0,26.537168
1,Bad,9.06,ab,45.0,NaN,38.384117
2,Abstellraum,2.57,ab,25.0,NaN,21.324510
3,Kochen,7.16,ab,45.0,NaN,38.384117
4,Wohnen,26.98,zu,NaN,3.0,39.805751
5,Zimmer_2,13.96,zu,NaN,2.0,26.537168
6,Zimmer_1,11.89,zu,NaN,2.0,26.537168
7,WC,1.45,ab,25.0,NaN,21.324510
8,Flur,9.07,--,NaN,NaN,0.000000


In [14]:
v = 2.0 # m/s
hour = 3600 # s

df_whg['d_min'] = np.sqrt(4*df_whg.dV/(hour*np.pi*v))
df_whg

,Raum,Flaeche,Typ,dV_min,f,dV,d_min
0,Schlafen,13.47,zu,NaN,2.0,26.537168,0.068504
1,Bad,9.06,ab,45.0,NaN,38.384117,0.082388
2,Abstellraum,2.57,ab,25.0,NaN,21.324510,0.061408
3,Kochen,7.16,ab,45.0,NaN,38.384117,0.082388
4,Wohnen,26.98,zu,NaN,3.0,39.805751,0.083900
5,Zimmer_2,13.96,zu,NaN,2.0,26.537168,0.068504
6,Zimmer_1,11.89,zu,NaN,2.0,26.537168,0.068504
7,WC,1.45,ab,25.0,NaN,21.324510,0.061408
8,Flur,9.07,--,NaN,NaN,0.000000,0.000000


In [15]:
d_g = [v*1e-3 for v in [80,100,80,100,100,80,80,80,0]]

df_whg['d_g'] = d_g
df_whg['v_g'] = (df_whg[df_whg.Typ!='--'].d_min/df_whg[df_whg.Typ!='--'].d_g)**2*v

df_whg

,Raum,Flaeche,Typ,dV_min,f,dV,d_min,d_g,v_g
0,Schlafen,13.47,zu,NaN,2.0,26.537168,0.068504,0.08,1.466500
1,Bad,9.06,ab,45.0,NaN,38.384117,0.082388,0.10,1.357560
2,Abstellraum,2.57,ab,25.0,NaN,21.324510,0.061408,0.08,1.178438
3,Kochen,7.16,ab,45.0,NaN,38.384117,0.082388,0.10,1.357560
4,Wohnen,26.98,zu,NaN,3.0,39.805751,0.083900,0.10,1.407840
5,Zimmer_2,13.96,zu,NaN,2.0,26.537168,0.068504,0.08,1.466500
6,Zimmer_1,11.89,zu,NaN,2.0,26.537168,0.068504,0.08,1.466500
7,WC,1.45,ab,25.0,NaN,21.324510,0.061408,0.08,1.178438
8,Flur,9.07,--,NaN,NaN,0.000000,0.000000,0.00,NaN
